# The Enumeration of Interactions

A *set of states* is a finite set $S$ with the base state $\ast \in S$, and an *interaction* on $S$ is a map $\phi: S \times S \to S \times S$ which satisfies that
$$
    \hat{\iota}(\phi(\hat{\iota}(\phi(s_1, s_2)))) = (s_1, s_2)
$$
where $\hat{\iota}(s_1, s_2) := (s_2, s_1)$ for any $s_1, s_2 \in S$ such that $\phi(s_1, s_2) \neq (s_1, s_2)$.

The function `enumerate_interactions` in the module `InteractionsEnumeration` enumerates all interactions $\phi$ with a number given as a size of the set $S$ of states.

In [1]:
import Pkg
Pkg.activate(".")
import Logging
Logging.disable_logging(Logging.Warn)

  Activating project at `c:\Users\User\LargeScaleInteractingSystems`


LogLevel(1001)

#### Required:
- [`JuMP` v0.22.3](https://jump.dev/)
- [`ConstraintSolver` v0.9.2](https://wikunia.github.io/ConstraintSolver.jl/stable/)

In [2]:
# include("RequiredPackages.jl")

In [3]:
# using modules
using LargeScaleInteractingSystems.States:
    StateSet
using LargeScaleInteractingSystems.Interactions:
    Interaction,
    interaction_plot
using LargeScaleInteractingSystems.ConservedQuantities:
    conserved_quantities
using LargeScaleInteractingSystems.InteractionsEnumeration:
    enumerate_interactions

precompile(enumerate_interactions, (StateSet, Int))
precompile(interaction_plot, (Interaction, ))

true

## First example; the case of $\#S = 2$
We will check that there are 18 interactions on $S$ the set $S = \{0 = \ast, 1\}$ of states.

In [4]:
# #S = 2
state_size = 2
state_set = StateSet(state_size)
interactions = enumerate_interactions(state_set)
println()
println("number of interactions: ", length(interactions))

# Variables: 40
# Constraints: 48
 - # Indicator: 16
 - # Reified: 32

  #Open    #Closed       Incumbent           Best Bound      Time [s] 
    2         0              -              0.0000000000       3.02   
    5         11             -              0.0000000000       6.58   
    7         21        0.0000000000        0.0000000000       6.80   
    6         32        0.0000000000        0.0000000000       6.90   
    5         45        0.0000000000        0.0000000000       6.91   
    5         55        0.0000000000        0.0000000000       6.92   
    5         65        0.0000000000        0.0000000000       6.92   
    4         76        0.0000000000        0.0000000000       6.93   
    5         87        0.0000000000        0.0000000000       6.93   
    3         97        0.0000000000        0.0000000000       6.94   
    2        108        0.0000000000        0.0000000000       6.94   
    6        118        0.0000000000        0.0000000000       6.94   
    2 

In [31]:
# display interactions
using Plots
plots = [interaction_plot(intr) for intr in interactions]
plot(plots..., layout = 18, size = (1500, 1200))

## Combinatorics

The number of interactions is explosively increasing with the number of states.

The lower and upper bound of the number of interactions with respect to the number of states are given by the following way.
First we consider the class $\mathcal{I}'$ of interactions $\phi: S \times S \to S \times S$ which satisfy that $\hat{\iota}(\phi(\hat{\iota}(\phi(s_1, s_2)))) = (s_1, s_2)$ for *arbitrary* $s_1, s_2 \in S$.
With composing $\phi$ with $\hat{\iota}$ we can regard any interaction $\phi$ in $\mathcal{I}'$ as a directed graph $G_\phi$ on $S \times S$, which is symmetry and of indegree and outdegree 1 (can include loops).
The perspective gives us that the number of interactions in $\mathcal{I}'$ coincides to the number of (unnecessary to be complete) matchings on the complete graph $K_{\#S^2}$, which is known as the telephone number $T(\#S^2)$.

Moreover, we can see that every general interaction $\phi$ can be considered as some interaction $\phi'$ in $\mathcal{I}'$ which does not include an edge $(s_1, s_2) \to (s_2, s_1)$ for $s_1, s_2 \in S$, replaced some loops $(s_1, s_2) \to (s_1, s_2)$ into edges $(s_1, s_2) \to (s_2, s_1)$.
Now let $M$ be the matching on $K_{\#S^2}$ which corresponds with the interaction $\phi'$.
Then the choice of loops $(s_1, s_2) \to (s_1, s_2)$ where $s_1 \neq s_2$ occurs variations of interactions.
If let $\chi_{\phi'}$ be the number of loops $(s_1, s_2) \to (s_1, s_2)$ where $s_1 \neq s_2$ then the number of variations for $\phi' \in \mathcal{I}'$ is given by $2^{\chi_{\phi'}}$.
As $\#S(\#S-1) < \chi_{\phi'} < \#S^2$ we have the lower and upper bound
$$
    \sum_{k=0}^{\lfloor n^2/2 \rfloor} 2^{\#S(\#S-1)-2k} \#\{\text{$2k$-matchings on $K_{\#S^2} \setminus \bigsqcup_{\frac{1}{2}\#S(\#S-1)} K_2$}\} < \#\{\text{interactions on $S$}\} < \sum_{k=0}^{\lfloor n^2/2 \rfloor} 2^{\#S^2-2k} \#\{\text{$2k$-matchings on $K_{\#S^2} \setminus \bigsqcup_{\frac{1}{2}\#S(\#S-1)} K_2$}\},
$$
where the graph $K_{\#S^2} \setminus \bigsqcup_{\frac{1}{2}\#S(\#S-1)} K_2$ is the complete graph $K_{\#S^2}$ without $k$ co-prime edges.

In [27]:
# calculate lower and upper bounds for the number of interactions

odd_factorial(k) = k <= 1 ? 1 : big(k)*odd_factorial(k-2)

# number of 2k-matchings on n vertices
MK(n, k) = binomial(big(n), big(2k)) * odd_factorial(2k-1)

# number of 2k-matchings on K_{n^2} - K_2 * n(n-1)/2
# with the inclusion-exclusion principle(TODO: check carefully)
M(n, k) = sum([(-1)^i * MK(n^2-2i, k-i) * binomial(big(div(n*(n-1),2)), big(i)) for i in 0:k])

lower_bound(n :: Int) =
    sum([M(n, k) * 2.0^(n^2 - n - 2k) for k in 0:div(n^2, 2)])

upper_bound(n :: Int) =
    sum([M(n, k) * 2^(n^2 - 2k) for k in 0:div(n^2, 2)])

for i in 2:10
    println(Float64(lower_bound(i)), " < {interactions on #S=", i, "} < ", Float64(upper_bound(i)))
end

9.5 < {interactions on #S=2} < 38.0
3006.0 < {interactions on #S=3} < 24048.0
6.59988105e7 < {interactions on #S=4} < 1.055980968e9
1.75154630393214e14 < {interactions on #S=5} < 5.604948172582848e15
8.695721733439377e22 < {interactions on #S=6} < 5.565261909401202e24
1.15723398516633e34 < {interactions on #S=7} < 1.4812595010129024e36
5.6036736513899486e47 < {interactions on #S=8} < 1.4345404547558268e50
1.287464694228741e64 < {interactions on #S=9} < 6.591819234451154e66
1.7744693163905044e83 < {interactions on #S=10} < 1.8170565799838765e86


## The case of $\#S = 3$

Let us try to enumerate interactions on $S = \{0 = \ast, 1, 2\}$.

In [30]:
# #S = 3
state_size = 3
state_set = StateSet(state_size)
interactions = enumerate_interactions(state_set)
# interactions = enumerate_interactions(state_set, 10) # if you set time limit
println()
println()
println("searched interactions: ", length(interactions))    # it costs about 14 minutes

# Variables: 180
# Constraints: 243
 - # Indicator: 81
 - # Reified: 162

  #Open    #Closed       Incumbent           Best Bound      Time [s] 
    2         0              -              0.0000000000      0.005   
    6         10             -              0.0000000000      0.010   
    9         19        0.0000000000        0.0000000000       0.01   
    10        32        0.0000000000        0.0000000000       0.02   
    14        42        0.0000000000        0.0000000000       0.02   
    16        54        0.0000000000        0.0000000000       0.03   
    16        64        0.0000000000        0.0000000000       0.04   
    7         79        0.0000000000        0.0000000000       0.05   
    7         91        0.0000000000        0.0000000000       0.06   
    16       102        0.0000000000        0.0000000000       0.06   
    8        116        0.0000000000        0.0000000000       0.08   
    9        127        0.0000000000        0.0000000000       0.10   
   


    10       1566       0.0000000000        0.0000000000       1.29   
    16       1576       0.0000000000        0.0000000000       1.29   
    12       1590       0.0000000000        0.0000000000       1.31   
    14       1600       0.0000000000        0.0000000000       1.32   
    8        1610       0.0000000000        0.0000000000       1.35   
    8        1620       0.0000000000        0.0000000000       1.37   
    12       1630       0.0000000000        0.0000000000       1.37   
    11       1641       0.0000000000        0.0000000000       1.38   
    10       1654       0.0000000000        0.0000000000       1.40   
    14       1664       0.0000000000        0.0000000000       1.40   
    16       1674       0.0000000000        0.0000000000       1.41   
    14       1684       0.0000000000        0.0000000000       1.42   
    11       1695       0.0000000000        0.0000000000       1.43   
    7        1705       0.0000000000        0.0000000000       1.43   
    1


    15       3167       0.0000000000        0.0000000000       2.53   
    14       3178       0.0000000000        0.0000000000       2.54   
    18       3188       0.0000000000        0.0000000000       2.55   
    12       3200       0.0000000000        0.0000000000       2.57   
    7        3211       0.0000000000        0.0000000000       2.58   
    17       3221       0.0000000000        0.0000000000       2.59   
    16       3232       0.0000000000        0.0000000000       2.60   
    11       3243       0.0000000000        0.0000000000       2.61   
    9        3255       0.0000000000        0.0000000000       2.63   
    13       3267       0.0000000000        0.0000000000       2.64   
    15       3277       0.0000000000        0.0000000000       2.65   
    15       3287       0.0000000000        0.0000000000       2.66   
    14       3300       0.0000000000        0.0000000000       2.67   
    15       3311       0.0000000000        0.0000000000       2.68   
    1


    9        4751       0.0000000000        0.0000000000       3.80   
    11       4761       0.0000000000        0.0000000000       3.81   
    9        4773       0.0000000000        0.0000000000       3.83   
    8        4784       0.0000000000        0.0000000000       3.84   
    11       4795       0.0000000000        0.0000000000       3.84   
    13       4805       0.0000000000        0.0000000000       3.85   
    17       4815       0.0000000000        0.0000000000       3.85   
    8        4828       0.0000000000        0.0000000000       3.86   
    12       4840       0.0000000000        0.0000000000       3.87   
    6        4854       0.0000000000        0.0000000000       3.88   
    14       4864       0.0000000000        0.0000000000       3.88   
    15       4875       0.0000000000        0.0000000000       3.89   
    14       4886       0.0000000000        0.0000000000       3.89   
    12       4896       0.0000000000        0.0000000000       3.90   
    1


    9        6333       0.0000000000        0.0000000000       5.07   
    10       6344       0.0000000000        0.0000000000       5.08   
    10       6354       0.0000000000        0.0000000000       5.09   
    8        6364       0.0000000000        0.0000000000       5.10   
    13       6375       0.0000000000        0.0000000000       5.10   
    13       6385       0.0000000000        0.0000000000       5.11   
    12       6398       0.0000000000        0.0000000000       5.12   
    11       6409       0.0000000000        0.0000000000       5.13   
    10       6422       0.0000000000        0.0000000000       5.14   
    14       6432       0.0000000000        0.0000000000       5.15   
    12       6442       0.0000000000        0.0000000000       5.16   
    15       6453       0.0000000000        0.0000000000       5.17   
    11       6463       0.0000000000        0.0000000000       5.18   
    12       6474       0.0000000000        0.0000000000       5.18   
    1

...


    14      33138       0.0000000000        0.0000000000      30.52   
    14      33148       0.0000000000        0.0000000000      30.53   
    11      33159       0.0000000000        0.0000000000      30.55   
    13      33169       0.0000000000        0.0000000000      30.56   
    11      33179       0.0000000000        0.0000000000      30.57   
    12      33190       0.0000000000        0.0000000000      30.58   
    14      33200       0.0000000000        0.0000000000      30.60   
    8       33210       0.0000000000        0.0000000000      30.61   
    13      33221       0.0000000000        0.0000000000      30.62   
    13      33231       0.0000000000        0.0000000000      30.63   
    14      33242       0.0000000000        0.0000000000      30.64   
    11      33253       0.0000000000        0.0000000000      30.66   
    15      33263       0.0000000000        0.0000000000      30.67   
    10      33274       0.0000000000        0.0000000000      30.68   
    1


    15      34695       0.0000000000        0.0000000000      32.23   
    10      34706       0.0000000000        0.0000000000      32.24   
    12      34716       0.0000000000        0.0000000000      32.25   
    12      34726       0.0000000000        0.0000000000      32.26   
    13      34737       0.0000000000        0.0000000000      32.27   
    8       34750       0.0000000000        0.0000000000      32.29   
    16      34760       0.0000000000        0.0000000000      32.30   
    11      34771       0.0000000000        0.0000000000      32.31   
    15      34781       0.0000000000        0.0000000000      32.32   
    13      34793       0.0000000000        0.0000000000      32.33   
    7       34805       0.0000000000        0.0000000000      32.34   
    15      34815       0.0000000000        0.0000000000      32.35   
    12      34828       0.0000000000        0.0000000000      32.36   
    18      34838       0.0000000000        0.0000000000      32.37   
    1

    14      36262       0.0000000000        0.0000000000      33.83   
    12      36272       0.0000000000        0.0000000000      33.84   
    11      36283       0.0000000000        0.0000000000      33.85   
    14      36294       0.0000000000        0.0000000000      33.87   
    9       36307       0.0000000000        0.0000000000      33.90   
    13      36317       0.0000000000        0.0000000000      33.91   
    11      36327       0.0000000000        0.0000000000      33.92   
    13      36337       0.0000000000        0.0000000000      33.93   
    13      36347       0.0000000000        0.0000000000      33.94   
    15      36357       0.0000000000        0.0000000000      33.95   
    13      36367       0.0000000000        0.0000000000      33.97   
    7       36379       0.0000000000        0.0000000000      33.99   
    14      36390       0.0000000000        0.0000000000      34.00   
    15      36401       0.0000000000        0.0000000000      34.01   
    11

Excessive output truncated after 524356 bytes.


    10      37838       0.0000000000        0.0000000000      35.47   
    7       37849       0.0000000000        0.0000000000      35.48   
    5       37859       0.0000000000        0.0000000000      35.50   
    13      37869       0.0000000000        0.0000000000      35.51   
    10      37880       0.0000000000        0.0000000000      35.52   
    10      37890       0.0000000000        0.0000000000      35.53   
    10      37900       0.0000000000        0.0000000000      35.54   
    6       37910       0.0000000000        0.0000000000      35.55   
    8       37920       0.0000000000        0.0000000000      35.57   
    15      37931       0.0000000000        0.0000000000      35.59   
    12      37948       0.0000000000        0.0000000000      35.60   
    18      37958       0.0000000000        0.0000000000      35.61   
    12      37970       0.0000000000        0.0000000000      35.63   
    10      37980       0.0000000000        0.0000000000      35.64   
    1

These are the initial 20 samples of interactions on $S = \{0 = \ast, 1, 2\}$.

In [ ]:
# display interactions
using Plots
n = 20
plots = [interaction_plot(interactions[i]) for i in 1:n]
plot(plots..., layout = n, size = (1500, 1200))

## References

[1] [Bannai, Kenichi and Kametani, Yukio and Sasada, Makiko. Topological Structures of Large Scale Interacting Systems via Uniform Functions and Forms. 2020.](https://arxiv.org/abs/2009.04699v4)